In [ ]:
import pandas as pd

# Load Qualtrics dataset
qualtrics_file = './dataset/EduRec_Raw.csv'
qualtrics_df = pd.read_csv(qualtrics_file)

# Step 1: Remove metadata/header rows and unusable data rows
qualtrics_df = qualtrics_df.iloc[2:].copy()

qualtrics_df['StartDate'] = pd.to_datetime(qualtrics_df['StartDate'], errors='coerce')
# Filter rows after 13 April 2025
qualtrics_df = qualtrics_df[qualtrics_df['StartDate'] > pd.Timestamp('2025-04-13')]
# Remove rows that are not completed
qualtrics_df = qualtrics_df[qualtrics_df['Progress'] == '100']
# Optional: reset index if needed
qualtrics_df.reset_index(drop=True, inplace=True)

qualtrics_df.rename(columns={
    "ResponseId": "Participant_Id",
    "Random ID": "Ref_Code",
    "Q3": "Digital_Familiarity",
    "Q4": "Portal_Comfort_Lvl",
    "Q5": "First_Sem",
    "Q7": "Freq_Usr",
    "Q6": "Course_BG",
    "Q78": "Local_Acclimated",
    "Q55": "Improvement",
    "Q56.1": "Issues",
    "Q57.1": "Comments"
}, inplace=True)


qualtrics_df.drop(columns=['Status', 'Q_RecaptchaScore', 'IPAddress', 'Finished', 'RecordedDate', 'RecipientLastName', 'RecipientFirstName', 'RecipientEmail', 'ExternalReference', 'LocationLatitude', 'LocationLongitude', 'DistributionChannel', 'UserLanguage','Consent'], inplace=True, errors='ignore')
qualtrics_df.drop(columns=['Q1', 'Q2', 'Q66', 'Q48', 'Q56', 'Q57', 'Q59', 'Q95', 'Q61', 'Q68', 'Q67', 'Q69', 'Q71', 'Q70', 'Q72'], inplace=True, errors='ignore')
# Step 2: Define task time question mappings (First Click and Page Submit columns)
task_time_questions = {
    81: ('Q81_First Click', 'Q81_Page Submit'),
    82: ('Q82_First Click', 'Q82_Page Submit'),
    83: ('Q83_First Click', 'Q83_Page Submit'),
    85: ('Q85_First Click', 'Q85_Page Submit'),
    60: ('Q60_First Click', 'Q60_Page Submit'),
    86: ('Q86_First Click', 'Q86_Page Submit'),
    87: ('Q87_First Click', 'Q87_Page Submit'),
    88: ('Q88_First Click', 'Q88_Page Submit'),
    89: ('Q89_First Click', 'Q89_Page Submit'),
    90: ('Q90_First Click', 'Q90_Page Submit'),
    91: ('Q91_First Click', 'Q91_Page Submit'),
    92: ('Q92_First Click', 'Q92_Page Submit')
}

# Task mapping as DataFrame
task_mapping = pd.DataFrame({
    'variant': ['A', 'A', 'A', 'B', 'B', 'B', 'C', 'C', 'C', 'D', 'D', 'D'],
    'task_type': ['view_class_info', 'view_personal_info', 'view_finance_info'] * 4,
    'task_qns': [81, 82, 83, 85, 60, 86, 87, 88, 89, 90, 91, 92]
})

# Step 3: Loop through task_time_questions and compute time
for task_qn, (first_click_col, submit_col) in task_time_questions.items():
    # Get variant and task_type
    row = task_mapping[task_mapping['task_qns'] == task_qn].iloc[0]
    variant = row['variant']
    task_type = row['task_type']

    # Create a descriptive column name
    col_name = f"{variant}_{task_type.replace(' ', '_')}_time_taken"

    # Compute task time and store as new column
    qualtrics_df[col_name] = (
        pd.to_numeric(qualtrics_df[submit_col], errors='coerce') -
        pd.to_numeric(qualtrics_df[first_click_col], errors='coerce')
    )
    
# Step 4: Define feedback questions (Qx_1 to Qx_7 per variant)
# Loop through each variant and compute feedback columns
for variant, q_prefix in {'A': 'Q45', 'B': 'Q46', 'C': 'Q47', 'D': 'Q50'}.items():
    feedback_cols = [f"{q_prefix}_{i}" for i in range(1, 8)]

    # Compute feedback avg and raw scores in original DataFrame
    qualtrics_df[f"{variant}_feedback_avg"] = qualtrics_df[feedback_cols].apply(pd.to_numeric, errors='coerce').mean(axis=1)
    qualtrics_df[f"{variant}_feedback_scores"] = qualtrics_df[feedback_cols].apply(lambda row: row.tolist(), axis=1)


prefixes_to_drop = ['Q6', 'Q45', 'Q46', 'Q47', 'Q50', 'Q81', 'Q82', 'Q83', 'Q85', 'Q60', 'Q86', 'Q87', 'Q88', 'Q89', 'Q90', 'Q91', 'Q92']  # <-- Add any other prefixes you want
cols_to_drop = [col for col in qualtrics_df.columns if any(col.startswith(prefix) for prefix in prefixes_to_drop)]
qualtrics_df.drop(columns=cols_to_drop, inplace=True)


print(qualtrics_df.columns.tolist())

qualtrics_df.to_csv('metric_qualtrics_wide.csv', index=False)

['StartDate', 'EndDate', 'Progress', 'Duration (in seconds)', 'Participant_Id', 'Digital_Familiarity', 'Portal_Comfort_Lvl', 'First_Sem', 'Freq_Usr', 'Local_Acclimated', 'Course_BG', 'Improvement', 'Issues', 'Comments', 'Ref_Code', 'timestamp', 'arrangement', 'A_view_class_info_time_taken', 'A_view_personal_info_time_taken', 'A_view_finance_info_time_taken', 'B_view_class_info_time_taken', 'B_view_personal_info_time_taken', 'B_view_finance_info_time_taken', 'C_view_class_info_time_taken', 'C_view_personal_info_time_taken', 'C_view_finance_info_time_taken', 'D_view_class_info_time_taken', 'D_view_personal_info_time_taken', 'D_view_finance_info_time_taken', 'A_feedback_avg', 'A_feedback_scores', 'B_feedback_avg', 'B_feedback_scores', 'C_feedback_avg', 'C_feedback_scores', 'D_feedback_avg', 'D_feedback_scores']
  Ref_Code                  timestamp
0    11096  2025-04-14T12:24:50+08:00
1    94531  2025-04-15T20:32:23+08:00
2    70610  2025-04-15T22:26:27+08:00
3    68221  2025-04-15T23:37